In [ ]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
print(tf.__version__)

2.15.0


In [ ]:
fashion_mnist = keras.datasets.fashion_mnist

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
train_images[0].shape

(28, 28)

In [ ]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [ ]:
def build_model(hp):
  model = keras.Sequential([
      keras.layers.Conv2D(
          filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
          kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
          activation='relu',
          input_shape=(28,28,1)
      ),
      keras.layers.Conv2D(
          filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
          kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
          activation='relu'
      ),
      keras.layers.Flatten(),
      keras.layers.Dense(
          units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
          activation='relu'
      ),
      keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.9070000052452087

Best val_accuracy So Far: 0.9100000262260437
Total elapsed time: 00h 03m 18s


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        25632     
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 48)                743472    
                                                                 
 dense_1 (Dense)             (None, 10)                490       
                                                                 
Total params: 769914 (2.94 MB)
Trainable params: 769914 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 4ms/step - loss: 0.1675 - accuracy: 0.9385 - val_loss: 0.2449 - val_accuracy: 0.9147
Epoch 5/10
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1325 - accuracy: 0.9502 - val_loss: 0.2693 - val_accuracy: 0.9108
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1019 - accuracy: 0.9614 - val_loss: 0.2866 - val_accuracy: 0.9142
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0771 - accuracy: 0.9720 - val_loss: 0.3183 - val_accuracy: 0.9162
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0636 - accuracy: 0.9762 - val_loss: 0.3493 - val_accuracy: 0.9125
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0478 - accuracy: 0.9824 - val_loss: 0.3907 - val_accuracy: 0.9100
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0390 - accuracy: 0.9859 - val_loss: 0.4115 - val_accuracy